# Real-World Data Model

### Import Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import itertools

### User Inputs and Preliminaries

In [2]:
stocks = ["HSBA.L", "RR.L", "BP.L", "BLND.L", "AV.L"]  # list of stocks
asset_count = len(stocks)

time_count = 5
risk_free_rate = 1
strike_price = 350
option_type = "call"
selected_time = 0

scenarios = []
if selected_time > 0:
    for i in range(asset_count):
        while True:
            try:
                scenario = list(map(int, input(f"Enter scenario for asset {i+1} up to time {selected_time} (use 1 for up and 0 for down): ").strip().split()))
                if all(val in (0, 1) for val in scenario) and len(scenario) == selected_time:
                    scenarios.append(scenario)
                    break
                else:
                    print(f"Invalid input. Please enter exactly {selected_time} integers consisting of 0s and 1s only.")
            except ValueError:
                print("Invalid input. Please enter integers (0 or 1) separated by spaces.")

if selected_time > 0:
    print(f"Scenarios for each asset up to time {selected_time}:", scenarios)

### Extracting Stock Data

In [3]:
stock_data = yf.download(" ".join(stocks), start="2023-01-01", end="2023-06-30", rounding=True)["Close"]
stock_data.to_csv("yf_stock_data.csv", index=True, header=True)

[*********************100%***********************]  5 of 5 completed


### Reading CSV File

In [4]:
raw_data = pd.DataFrame(stock_data)
raw_data.head()

,AV.L,BLND.L,BP.L,HSBA.L,RR.L
Date,,,,,
2023-01-03,448.9,408.2,483.35,529.9,98.91
2023-01-04,456.8,412.8,465.85,543.5,101.40
2023-01-05,448.7,409.5,471.75,565.3,102.66
2023-01-06,456.0,410.2,477.05,568.6,102.90
2023-01-09,455.0,410.0,479.30,563.2,103.76


### Data Pre-processing

In [5]:
raw_data.fillna(method='ffill', inplace=True)
clean_data = raw_data
clean_data.head()

,AV.L,BLND.L,BP.L,HSBA.L,RR.L
Date,,,,,
2023-01-03,448.9,408.2,483.35,529.9,98.91
2023-01-04,456.8,412.8,465.85,543.5,101.40
2023-01-05,448.7,409.5,471.75,565.3,102.66
2023-01-06,456.0,410.2,477.05,568.6,102.90
2023-01-09,455.0,410.0,479.30,563.2,103.76


### Finding Initial Prices and Factors

In [6]:
initial_prices = []
up_factors = []
down_factors = []

delta_t = 1

for stock in stocks:
    clean_data[f'{stock}_Log_Return'] = np.log(clean_data[stock] / clean_data[stock].shift(1))

    st_dev = clean_data[f'{stock}_Log_Return'].std()

    U = round(np.exp(st_dev * np.sqrt(delta_t)), 3)
    D = round(np.exp(-st_dev * np.sqrt(delta_t)), 3)

    initial_prices.append(clean_data[stock].iloc[-1])
    up_factors.append(U)
    down_factors.append(D)

print("Initial Prices:", initial_prices)
print("Up:", up_factors)
print("Down:", down_factors)

Initial Prices: [618.8, 148.7, 454.8, 300.6, 388.3]
Up: [1.015, 1.03, 1.02, 1.018, 1.015]
Down: [0.985, 0.971, 0.98, 0.982, 0.986]


### Producing "I"

In [7]:
set_i = list(range(asset_count + 1))
I_factor = time_count - selected_time

set_I = list(itertools.product(set_i, repeat=I_factor))
print(set_I)

[(0, 0, 0, 0, 0), (0, 0, 0, 0, 1), (0, 0, 0, 0, 2), (0, 0, 0, 0, 3), (0, 0, 0, 0, 4), (0, 0, 0, 0, 5), (0, 0, 0, 1, 0), (0, 0, 0, 1, 1), (0, 0, 0, 1, 2), (0, 0, 0, 1, 3), (0, 0, 0, 1, 4), (0, 0, 0, 1, 5), (0, 0, 0, 2, 0), (0, 0, 0, 2, 1), (0, 0, 0, 2, 2), (0, 0, 0, 2, 3), (0, 0, 0, 2, 4), (0, 0, 0, 2, 5), (0, 0, 0, 3, 0), (0, 0, 0, 3, 1), (0, 0, 0, 3, 2), (0, 0, 0, 3, 3), (0, 0, 0, 3, 4), (0, 0, 0, 3, 5), (0, 0, 0, 4, 0), (0, 0, 0, 4, 1), (0, 0, 0, 4, 2), (0, 0, 0, 4, 3), (0, 0, 0, 4, 4), (0, 0, 0, 4, 5), (0, 0, 0, 5, 0), (0, 0, 0, 5, 1), (0, 0, 0, 5, 2), (0, 0, 0, 5, 3), (0, 0, 0, 5, 4), (0, 0, 0, 5, 5), (0, 0, 1, 0, 0), (0, 0, 1, 0, 1), (0, 0, 1, 0, 2), (0, 0, 1, 0, 3), (0, 0, 1, 0, 4), (0, 0, 1, 0, 5), (0, 0, 1, 1, 0), (0, 0, 1, 1, 1), (0, 0, 1, 1, 2), (0, 0, 1, 1, 3), (0, 0, 1, 1, 4), (0, 0, 1, 1, 5), (0, 0, 1, 2, 0), (0, 0, 1, 2, 1), (0, 0, 1, 2, 2), (0, 0, 1, 2, 3), (0, 0, 1, 2, 4), (0, 0, 1, 2, 5), (0, 0, 1, 3, 0), (0, 0, 1, 3, 1), (0, 0, 1, 3, 2), (0, 0, 1, 3, 3), (0, 0, 1, 3, 

### Coefficients b0, b1, ..., bm

In [8]:
combined = [(i, (risk_free_rate - d) / (u - d) if (u - d) else 0, s, u, d)
            for i, (s, u, d) in enumerate(zip(initial_prices, up_factors, down_factors))]

combined.insert(0, (0, 1, 0, 0, 0))
combined.sort(key=lambda x: x[1], reverse=True)
indices, coefficients, initial_prices, up_factors, down_factors = map(list, zip(*combined))

# Use the sorted indices to rearrange scenarios list
if selected_time > 0:
    scenarios = [scenarios[i] for i in indices[1:]]

initial_prices.pop(0)
up_factors.pop(0)
down_factors.pop(0)

print("Sorted Coefficients:", coefficients)
print("Sorted initial_prices:", initial_prices)
print("Sorted up_factors:", up_factors)
print("Sorted down_factors:", down_factors)
if selected_time > 0:
    print(f"Scenarios for each asset up to time {selected_time} after sorting:", scenarios)

Sorted Coefficients: [1, 0.5000000000000019, 0.5, 0.5, 0.4915254237288136, 0.482758620689657]
Sorted initial_prices: [618.8, 454.8, 300.6, 148.7, 388.3]
Sorted up_factors: [1.015, 1.02, 1.018, 1.03, 1.015]
Sorted down_factors: [0.985, 0.98, 0.982, 0.971, 0.986]


### Outcomes μ0, μ1, ..., μm

In [9]:
mu = [(1,) * i + (0,) * (asset_count - i) for i in range(asset_count + 1)]
print(mu)

[(0, 0, 0, 0, 0), (1, 0, 0, 0, 0), (1, 1, 0, 0, 0), (1, 1, 1, 0, 0), (1, 1, 1, 1, 0), (1, 1, 1, 1, 1)]


### Outcome Probabilities P(μ0), P(μ1), ..., P(μm)

In [10]:
mu_probabilities = [coefficients[i] - coefficients[i+1] for i in range(len(coefficients)-1)] + [coefficients[-1]]
print(mu_probabilities)

[0.4999999999999981, 1.887379141862766e-15, 0.0, 0.008474576271186418, 0.008766803039156557, 0.482758620689657]


### Finding all Outcome Matrices

In [11]:
all_matrices = [np.column_stack([mu[i] for i in indices]) for indices in set_I]

if selected_time > 0:
    scenarios_np = np.array(scenarios)
    all_matrices = [np.hstack([scenarios_np, matrix]) for matrix in all_matrices]

for idx, matrix in enumerate(all_matrices):
    print(f"X {idx + 1}:\n {matrix}\n")

X 1:
 [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

X 2:
 [[0 0 0 0 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

X 3:
 [[0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

X 4:
 [[0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]
 [0 0 0 0 0]]

X 5:
 [[0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]

X 6:
 [[0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]]

X 7:
 [[0 0 0 1 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

X 8:
 [[0 0 0 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

X 9:
 [[0 0 0 1 1]
 [0 0 0 0 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

X 10:
 [[0 0 0 1 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]
 [0 0 0 0 0]]

X 11:
 [[0 0 0 1 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 0]]

X 12:
 [[0 0 0 1 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]]

X 13:
 [[0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

X 14:
 [[0 0 0 1 1]
 [0 0 0 1 0]
 [0 0 0 0 0]
 

### Calculating Payoffs

In [12]:
payoffs = []
for matrix in all_matrices:
    S_t = initial_prices.copy()
    for t in range(len(matrix[0])):
        for i in range(asset_count):
            multiplier = up_factors[i] if matrix[i][t] == 1 else down_factors[i]
            S_t[i] *= multiplier
    payoff = max((sum(S_t) / asset_count) - strike_price, 0) if option_type == "call" else max(
        strike_price - (sum(S_t) / asset_count), 0)
    payoffs.append(payoff)

print(payoffs)

[0, 3.4130205574915067, 6.768964870835475, 8.781611006860203, 10.341414524646893, 12.47005856831646, 3.4130205574915067, 7.0144614309309645, 10.37040574427499, 12.383051880299718, 13.942855398086408, 16.071499441755975, 6.768964870835475, 10.37040574427499, 13.863327376531004, 15.875973512555731, 17.43577703034242, 19.56442107401199, 8.781611006860203, 12.383051880299718, 15.875973512555731, 17.962403009982495, 19.522206527769242, 21.65085057143881, 10.341414524646893, 13.942855398086408, 17.43577703034242, 19.522206527769242, 21.17678698432985, 23.30543102799936, 12.47005856831646, 16.071499441755975, 19.56442107401199, 21.65085057143881, 23.30543102799936, 25.49668224942394, 3.41302055749145, 7.0144614309309645, 10.37040574427499, 12.38305188029966, 13.942855398086408, 16.07149944175592, 7.0144614309309645, 10.72559085889145, 14.081535172235476, 16.094181308260147, 17.653984826046894, 19.782628869716405, 10.37040574427499, 14.081535172235476, 17.57445680449149, 19.587102940516218, 21

### Calculating Cmax

In [13]:
c_max = sum([payoffs[idx] * np.prod([mu_probabilities[i] for i in indices]) for idx, indices in enumerate(set_I)])

if selected_time == 0:
    print(f"Price of option at time {selected_time}:", "\n\tC_max = £", round(c_max, 2))
elif selected_time > 0:
    print(f"Value of option at time {selected_time}:", "\n\tCmax = £", round(c_max, 2))
else:
    raise ValueError("Selected time >= 0 must be satisfied!")

Price of option at time 0: 
	C_max = £ 32.24
